# Question 5: Primative Abundant Numbers

## 5(a) Inital investigation

The sum of divisors of a nautural number, denoted $\sigma$ is a function $\sigma:\mathbb{N}\to\mathbb{N}$ which maps $n$ to the sum of it's divisors,
$$
    \sigma(n) = \displaystyle\sum_{d\mid n} d.
$$
An abundant number is a natural number whose sum of divisors is greater than 2 times the number, i.e. $\sigma(n) > 2n$. A deficient number is a number whose sum of divisors is less than 2 times the number, i.e. $\sigma(n) < 2n$. A primative abundant number (PAN) is an abundant number, whose proper divisors are all deficient numbers.

20's divisors are 1, 2, 4, 5, 10, 20. Hence, $\sigma(20) = 1 + 2 + 4 + 5 + 10 + 20 = 42 > 2(20)$. Hence, 20 is an abundant number. $\sigma(1) = 1 < 2(1)$, $\sigma(2) = 1 + 2 = 3 < 2(4)$, $\sigma(4) = 1 + 2 + 4 = 7 < 2(4)$, $\sigma(5) = 1 + 5 = 6 < 2(5)$, and $\sigma(10) = 1 + 2 + 5 + 10 = 18 < 2(10)$ (20 is not a proper divisors); hence, 1, 2, 4, 5 and 10 are all deficient.

18's divisors are 1, 2, 3, 6, 9, 18. Hence, $\sigma(18) = 39 > 2(18)$. Therefore, 18 is an abundant number. However, 6's divisors are 1, 2, 3, 9, so $\sigma(6) = 12 = 2(6)$, which means 6 is not a deficient number (in fact its a perfect number!). Therefore, 18 is an abundant number, but not a primitive abundant number.

$\sigma(19)=20$, $\sigma(21)=32$, $\sigma(22)=36$, and $\sigma(23)=24$. All of these numbers are defficent themselves, and are therefore not PANs.

## 5(b) Print all the PANs less than 100

In [10]:
using Primes

@enum numberType undefined=-1 deficient=0 perfect=1 abundant=2 primitiveabundant=3
function findPANs(upperBound::Integer, findDivisors::Function=divisors)::Vector{Integer}
    Type = Dict(n => undefined for n in 1:upperBound)
    for n in keys(Type)
        sigma = sum(findDivisors(n))
        if sigma < 2*n
            Type[n] = deficient
        elseif sigma > 2*n
            Type[n] = abundant
        else
            Type[n] = perfect
        end
    end
    
    for n in keys(Type)
        if Type[n] != abundant
            continue
        end
    
        for properDivisors in findDivisors(n)[1:end-1]  # the last element is the number itself, which is not a proper divisor
            if Type[properDivisors] != deficient
                @goto nextNumber
            end
        end
    
        Type[n] = primitiveabundant
        @label nextNumber
    end
    
    return [n for n in sort(collect(keys(Type))) if Type[n] == primitiveabundant]
end

print(findPANs(1000))

Integer[20, 70, 88, 104, 272, 304, 368, 464, 550, 572, 650, 748, 836, 945]

When I was thinking about writing this code, I imagined myself ranging `n` between 1 and 1000, generating its divisors, calculating its sigma, and then recursively calculating the sigma's for each of its proper divisors, testing are they deficient or not. I felt like this was too computationally inefficient for my liking. Instead, I decided to do something a little more interesting, and generate each number's divisors exactly once, then caching it's "type" (deficient, perfect, abundant) in a hashtable; then, I'd have $\mathcal{O}(1)$ access to each number's type.

Now that I have categorised each number in a hashtable, I can very quickly loop through all the numbers, check "is this number abundant?" (if not, `continue`) If it is, I can check, "are each of your proper divisors categorised as deficient?" (If not, `@goto nextNumber`) If it is, reclassify (or specfiy its classication, rather) as a primative abundant number.

The idea was 100% mine! I did have some help with a couple things:

1. Though I was familiar with `enum`s, coming from C/C++, I wasn't sure how to define and use them in Julia. I had an LLM explain it to me.

2. Breaking/continue the outer loop from an inner loop is always a pain. Some languages have keywords that do that! Julia does not, and I learned how to use `@goto escape` from this StackOverflow thread ([isebarn, 2016](https://stackoverflow.com/a/39797777))

3. In the final part of the code, initally, I had `for n in keys(numbers)`, but it was printing the PANs in a seemingly random order. An LLM explained to me that `Dicts` are unordered, and output keys to me in order of their hashes, not the values themselves (something about putting the hashes in buckets?). Anyway, no problem I thought, `for n in sort(keys(numbers))`, but that's how I learned that `keys(numbers)` returns a generator, and `sort` doesn't accept generators as an argument (this is contrary to Python, for example, where you can pass iterables to standard functions, generally, no problem). The LLM helped me learn that the function `collect` allows me to collect all the entries in a generator, and return them in a vector. Hence, the final solution had the line `for n in sort(collect(keys(number)))` (The "final part" printing part of the code has been ommited, now that I have function-ified `findPANs`, to make 5(c) easier)

## 5(c) Write your own `divisors` function

I'm going to function-ify my previous solution, but let it accept a function pointer to a function which takes a number and returns a vector of numbers, that way I can reuse my previous solution, while passing `mks_divisors` instead of the `Primes.divisors`.

In [16]:
function mks_divisors(n::Integer)::Vector{Integer}
  return [i for i in 1:n if n%i == 0]
end

print(findPANs(1000, mks_divisors))

Integer[20, 70, 88, 104, 272, 304, 368, 464, 550, 572, 650, 748, 836, 945]

I KNOW I KNOW I KNOW, the guy who was so concerned about computational efficiency, that he used a hashtable in the previous question just wrote *that*. Maybe I like the juxtaposition ;)

My beautiful horrible very bad code uses an array comprehension to allow `i` to range between 1 and a given number, `n`, and only adds it to the list if $n\pmod{i} \equiv 0 \iff i\mid n$.

This is quite possibly the most naiive solution to this problem. Right of the rip, a signifncalty better solution would only check `i in 1:sqrt(n)` and then append `n` at the end.

I suspect an even better solution might look like a combination of a hashtable/caching and the Sieve of Eratosthenes. For example, I could imagine a good soltuion to this might:
1. start at 1, see that it doesn't have itself as a divisor. Append itself, to it's own, and each of its multiple's cached list.
2. move on to 2. See that it doesn't have itself as a divisor. To it's list of divisors, appead itself, then go through all it's multiples and append a 2 to their list too.
3. repeat until the final case sees us appending `upperBound=1000` to it's own divisors list.

Describing it got me excited to implement it, but we'll have to see if I have the nerves to commit to that.